In [1]:
OUT_DIR = '/tmp/'
NUM_WORKERS = 16
BATCH_SIZE = 128


# Barebones starter example

In [2]:
from robustness import model_utils, datasets, train, defaults
from robustness.datasets import FashionMnist#CIFAR#FashionMnist
from robustness import data_augmentation as da
import torch 
import torchvision.datasets

#import sys
#sys.path.append('/home/u21010246/mlpr/venv/lib/python3.8/site-packages/robustness')

# We use cox (http://github.com/MadryLab/cox) to log, store and analyze
# results. Read more at https//cox.readthedocs.io.
from cox.utils import Parameters
import cox.store


/home/u21010246/.conda/envs/rob/lib/python3.9/site-packages/robustness/train.py:24: UserWarning: Could not import amp.
  warnings.warn('Could not import amp.')


### Make dataset and loaders

In [4]:
# Hard-coded dataset, architecture, batch size, workers
#ds = FashionMnist('/tmp/fashion-mnist/data') # CIFAR('/tmp')
#m, _ = model_utils.make_and_restore_model(arch='resnet18', dataset=ds)
#train_loader, val_loader = ds.make_loaders(batch_size=BATCH_SIZE, workers=NUM_WORKERS)

fm_train = torchvision.datasets.FashionMNIST('/tmp', download=True, train=True,transform=da.TRAIN_TRANSFORMS_DEFAULT(32))
fm_val = torchvision.datasets.FashionMNIST('/tmp', download=True, train=False,transform=da.TEST_TRANSFORMS_DEFAULT(32))

train_loader = torch.utils.data.DataLoader(fm_train, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
val_loader = torch.utils.data.DataLoader(fm_val, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)

ds = FashionMnist('/tmp/FashionMNIST/raw') # CIFAR('/tmp')
m, _ = model_utils.make_and_restore_model(arch='resnet18', dataset=ds)

### Make a cox store for logging

In [5]:
# Create a cox store for logging
out_store = cox.store.Store(OUT_DIR)

Logging in: /tmp/04290d4a-ec86-48d6-bd11-3d8ce7c2a298


### Set up training arguments

In [6]:
# Hard-coded base parameters
train_kwargs = {
    'out_dir': "train_out",
    'adv_train': 1,
    'constraint': '2',
    'eps': 0.5,
    'attack_lr': 0.1,
    'attack_steps': 7,
    'epochs': 120
}
train_args = Parameters(train_kwargs)

# Fill whatever parameters are missing from the defaults
train_args = defaults.check_and_fill_args(train_args,
                        defaults.TRAINING_ARGS, FashionMnist)
train_args = defaults.check_and_fill_args(train_args,
                        defaults.PGD_ARGS, FashionMnist)

### Train Model

In [7]:
# Train a model
train.train_model(train_args, m, (train_loader, val_loader), store=out_store)
pass

Train Epoch:0 | Loss 1.2911 | AdvPrec1 54.963 | AdvPrec5 94.125 | Reg term: 0.0 
Val Epoch:0 | Loss 0.7215 | NatPrec1 72.610 | NatPrec5 99.230 | Reg term: 0.0 ||
Val Epoch:0 | Loss 0.8764 | AdvPrec1 66.130 | AdvPrec5 98.980 | Reg term: 0.0 ||
Train Epoch:1 | Loss 0.7093 | AdvPrec1 71.245 | AdvPrec5 99.363 | Reg term: 0.0 
Train Epoch:2 | Loss 0.6198 | AdvPrec1 74.830 | AdvPrec5 99.585 | Reg term: 0.0 


BlockingIOError: [Errno 11] Resource temporarily unavailable

# Customizations

## Custom loss

In [7]:
train_crit = ch.nn.CrossEntropyLoss()
def custom_train_loss(logits, targ):
    probs = ch.ones_like(logits) * 0.5
    logits_to_multiply = ch.bernoulli(probs) * 9 + 1
    return train_crit(logits_to_multiply * logits, targ)

adv_crit = ch.nn.CrossEntropyLoss(reduction='none').cuda()
def custom_adv_loss(model, inp, targ):
    logits = model(inp)
    probs = ch.ones_like(logits) * 0.5
    logits_to_multiply = ch.bernoulli(probs) * 9 + 1
    new_logits = logits_to_multiply * logits
    return adv_crit(new_logits, targ), new_logits

train_args.custom_train_loss = custom_train_loss
train_args.custom_adv_loss = custom_adv_loss

In [8]:
train.train_model(train_args, m, (train_loader, val_loader), store=out_store)

Train Epoch:0 | Loss 2.4313 | AdvPrec1 14.508 | AdvPrec5 62.072 | Reg term: 0.0 ||: 100%|█| 
Val Epoch:0 | Loss 2.1306 | NatPrec1 21.650 | NatPrec5 78.680 | Reg term: 0.0 ||: 100%|█| 20
Val Epoch:0 | Loss 2.1866 | AdvPrec1 19.940 | AdvPrec5 74.620 | Reg term: 0.0 ||: 100%|█| 20
Train Epoch:1 | Loss 2.1328 | AdvPrec1 21.886 | AdvPrec5 77.114 | Reg term: 0.0 ||: 100%|█| 
Train Epoch:2 | Loss 2.0693 | AdvPrec1 25.616 | AdvPrec5 80.604 | Reg term: 0.0 ||: 100%|█| 
Train Epoch:3 | Loss 2.0280 | AdvPrec1 29.024 | AdvPrec5 83.142 | Reg term: 0.0 ||: 100%|█| 
Train Epoch:4 | Loss 1.9914 | AdvPrec1 31.522 | AdvPrec5 84.474 | Reg term: 0.0 ||: 100%|█| 
Val Epoch:4 | Loss 1.7850 | NatPrec1 42.190 | NatPrec5 91.360 | Reg term: 0.0 ||: 100%|█| 20
Val Epoch:4 | Loss 1.9505 | AdvPrec1 34.010 | AdvPrec5 87.060 | Reg term: 0.0 ||: 100%|█| 20


DataParallel(
  (module): AttackerModel(
    (normalizer): InputNormalize()
    (model): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (layer1): SequentialWithArgs(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (shortcut): Sequential()
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     

## Custom data loaders

### Using LambdaLoader

In [ ]:
from robustness.loaders import LambdaLoader

def label_noiser(ims, labels):
    label_noise = ch.randint_like(labels, high=9)
    probs = ch.ones_like(label_noise) * 0.1
    labels_to_noise = ch.bernoulli(probs.float()).long()
    new_labels = (labels + label_noise * labels_to_noise) % 10
    return ims, new_labels

train_loader = LambdaLoader(train_loader, label_noiser)

In [ ]:
train.train_model(train_args, m, (train_loader, val_loader), store=out_store)
pass

### Using TransformedLoader

In [ ]:
from robustness.loaders import TransformedLoader
from robustness.data_augmentation import TRAIN_TRANSFORMS_DEFAULT

def make_rand_labels(ims, targs):
    new_targs = ch.randint(0, high=10,size=targs.shape).long()
    return ims, new_targs

train_loader_transformed = TransformedLoader(train_loader,
                                            make_rand_labels,
                                            TRAIN_TRANSFORMS_DEFAULT(32),
                                            workers=8,
                                            batch_size=BATCH_SIZE,
                                            do_tqdm=True)

In [ ]:
train.train_model(train_args, m, (train_loader, val_loader), store=out_store)
pass

## Custom per-iteration logging

In [ ]:
CUSTOM_SCHEMA = {'iteration': int, 'weight_norm': float }
out_store.add_table('custom', CUSTOM_SCHEMA)

In [ ]:
from torch.nn.utils import parameters_to_vector as flatten

def log_norm(mod, it, loop_type, inp, targ):
    if loop_type == 'train':
        curr_params = flatten(mod.parameters())
        log_info_custom = { 'iteration': it,
                    'weight_norm': ch.norm(curr_params).detach().cpu().numpy() }
        out_store['custom'].append_row(log_info_custom)
    
train_args.iteration_hook = log_norm

In [ ]:
train.train_model(train_args, m, (train_loader, val_loader), store=out_store)
pass

## Custom architecture

In [ ]:
from torch import nn
from robustness.model_utils import make_and_restore_model

class MLP(nn.Module):
    # Must implement the num_classes argument
    def __init__(self, num_classes=10):
        super().__init__()
        self.fc1 = nn.Linear(32*32*3, 1000)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(1000, num_classes)

    def forward(self, x, *args, **kwargs):
        out = x.view(x.shape[0], -1)
        out = self.fc1(out)
        out = self.relu1(out)
        return self.fc2(out)

new_model = MLP(num_classes=10)

In [ ]:
new_model, _ = make_and_restore_model(arch=new_model, dataset=ds)

In [ ]:
train.train_model(train_args, new_model, (train_loader, val_loader), store=out_store)
pass